# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [30]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os
import re

In [31]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {
    "proceeding": {
        "file" : "proceedings.bib",
        "venuekey": "booktitle",
        "venue-pretext": "In the proceedings of ",
        "collection" : {"name":"publications",
                        "permalink":"/publication/"}
        
    },
    "journal":{
        "file": "pubs.bib",
        "venuekey" : "journal",
        "venue-pretext" : "",
        "collection" : {"name":"publications",
                        "permalink":"/publication/"}
    } 
}

In [32]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [ ]:
for pubsource in publist:
    parser = bibtex.Parser()
    bibdata = parser.parse_file(publist[pubsource]["file"])

    #loop through the individual references in a given bibtex file
    for bib_id in bibdata.entries:
        #reset default date
        pub_year = "1900"
        pub_month = "01"
        pub_day = "01"
        
        b = bibdata.entries[bib_id].fields
        
        try:
            pub_year = f'{b["year"]}'

            #todo: this hack for month and day needs some cleanup
            if "month" in b.keys(): 
                if(len(b["month"])<3):
                    pub_month = "0"+b["month"]
                    pub_month = pub_month[-2:]
                elif(b["month"] not in range(12)):
                    tmnth = strptime(b["month"][:3],'%b').tm_mon   
                    pub_month = "{:02d}".format(tmnth) 
                else:
                    pub_month = str(b["month"])
            if "day" in b.keys(): 
                pub_day = str(b["day"])

                
            pub_date = pub_year+"-"+pub_month+"-"+pub_day
            
            #strip out {} as needed (some bibtex entries that maintain formatting)
            clean_title = b["title"].replace("{", "").replace("}","").replace("\\","").replace(" ","-")    

            url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
            url_slug = url_slug.replace("--","-")

            md_filename = (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
            html_filename = (str(pub_date) + "-" + url_slug).replace("--","-")

            #Build Citation from text
            citation = ""

            #Create author list in ACS format with hack to make my name bolded
            target_last_name = "Smythe"
            author_list = []
            
            # Function to extract initials from names
            ##NOTE: this is still not working correctly and is not extracting the middle initial, likely due to how pybtex parses names
            import re
            def extract_initials(first_names):
                initials = []
                for name in first_names:
                    for part in name.strip().split():
                        if part:  # skip empty strings
                            initials.append(part[0])
                return ". ".join(initials) + "."

            for author in bibdata.entries[bib_id].persons["author"]:
                initials = extract_initials(author.first_names)
                last_name = " ".join(author.last_names)
                formatted_name = f"{last_name}, {initials}"

                if last_name == target_last_name:
                    formatted_name = f"**{formatted_name}**"

                author_list.append(formatted_name)

            # If there are multiple authors, join them with semicolons
            citation = "; ".join(author_list)

            #citation title // note: removed the quotations around the title
            citation += " " + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + "."

            #add venue logic depending on citation type
            venue = publist[pubsource]["venue-pretext"]+b[publist[pubsource]["venuekey"]].replace("{", "").replace("}","").replace("\\","")

            citation += f" <i>{venue}</i>" ## note: added italics to venue
            citation += f" <b>{pub_year}</b>" ## note: added bold to year

            #add volume, number, pages, doi to the citation output
            if "volume" in b:
                citation += f", <i>{b['volume']}</i>"
            if "number" in b:
                citation += f" ({b['number']})"
            if "pages" in b:
                citation += f", {b['pages']}"
            if "doi" in b:
                citation += f". DOI: {b['doi']}"
            else:
                citation += "."

            
            ## YAML variables // note: removed the quotations around the title
            md = "---\ntitle: " + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + "\n"

            md += """collection: """ +  publist[pubsource]["collection"]["name"]

            md += """\npermalink: """ + publist[pubsource]["collection"]["permalink"]  + html_filename
            
            ## category logic
            if publist[pubsource]["file"].endswith("proceedings.bib"):
                md += "\ncategory: proceedings"
            elif publist[pubsource]["file"].endswith("pubs.bib"):
                md += "\ncategory: manuscripts"

            note = False
            if "note" in b.keys():
                if len(str(b["note"])) > 5:
                    md += "\nexcerpt: '" + html_escape(b["note"]) + "'"
                    note = True

            md += "\ndate: " + str(pub_date) 

            md += "\nvenue: '" + html_escape(venue) + "'"
            
            ## note: changed paperurl to link
            url = False
            if "url" in b.keys():
                if len(str(b["url"])) > 5:
                    md += "\nlink: '" + b["url"] + "'"
                    url = True

            md += "\ncitation: '" + html_escape(citation) + "'"

            md += "\n---"

            
            ## Markdown description for individual page
            if note:
                md += "\n" + html_escape(b["note"]) + "\n"

            if url:
                md += "\n[Access paper here](" + b["url"] + "){:target=\"_blank\"}\n" 
            else:
                md += "\nUse [Google Scholar](https://scholar.google.com/scholar?q="+html.escape(clean_title.replace("-","+"))+"){:target=\"_blank\"} for full citation"

            md_filename = os.path.basename(md_filename)

            with open("../_publications/" + md_filename, 'w', encoding="utf-8") as f:
                f.write(md)
            print(f'SUCESSFULLY PARSED {bib_id}: \"', b["title"][:60],"..."*(len(b['title'])>60),"\"")
        # field may not exist for a reference
        except KeyError as e:
            print(f'WARNING Missing Expected Field {e} from entry {bib_id}: \"', b["title"][:30],"..."*(len(b['title'])>30),"\"")
            continue


Parsed: Chu, S.
Parsed: Solla, S.
Parsed: **Smythe, T.**
Parsed: Eng, M.
Parsed: Lavoie, R.
Parsed: Letcher, R.
SUCESSFULLY PARSED Chu_2025: " Per- and polyfluoroalkyl substance profiles revealed by targ ... "
Parsed: **Smythe, T.**
Parsed: Gauthier, L.
Parsed: Letcher, R.
SUCESSFULLY PARSED Smythe_2024: " Dietary and terrestrial exposure to methoxylated polybromina ... "
Parsed: Sesin, V.
Parsed: Judy, J.
Parsed: Kapustka, L.
Parsed: Opeolu, B.
Parsed: Ottinger, M.
Parsed: Bertsch, P.
Parsed: Wang, Y.
Parsed: Lazorchak, J.
Parsed: **Smythe, T.**
Parsed: Stahl, R.
SUCESSFULLY PARSED Sesin_2022: " The Importance of Fostering and Funding Scientific Research, ... "
Parsed: **Smythe, T.**
Parsed: Su, G.
Parsed: Bergman, Å.
Parsed: Letcher, R.
SUCESSFULLY PARSED Smythe_2022: " Metabolic transformation of environmentally-relevant bromina ... "
Parsed: **Smythe, T.**
Parsed: Mattioli, L.
Parsed: Letcher, R.
SUCESSFULLY PARSED Smythe2020: " Distribution behaviour in body compartments and in ov